In [19]:
import os
import torch
import shutil
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import f1_score, accuracy_score

In [20]:
base_path = '/home/pervinco/upstage-cv-classification-cv7'
weight_path = f'{base_path}/runs/best_9720'
submission_path = f"{weight_path}/preds/tta_submission.csv"

dataset_path = f'{base_path}/dataset/valid_dataset.csv'
incorrect_path = f'{weight_path}/incorrect'

In [21]:
valid_df = pd.read_csv(dataset_path)
submission_df = pd.read_csv(submission_path)

In [22]:
if os.path.exists(incorrect_path):
    shutil.rmtree(incorrect_path)
os.makedirs(incorrect_path)

In [23]:
# 예측 결과 비교 및 틀린 이미지 저장
valid_df['correct'] = valid_df['target'] == submission_df['target']
incorrect_images = valid_df[~valid_df['correct']]

for _, row in incorrect_images.iterrows():
    src_path = f"{base_path}/dataset/test/{row['ID']}"
    predicted_class = submission_df.loc[submission_df['ID'] == row['ID'], 'target'].values[0]
    dst_filename = f"pred{predicted_class}_gt{row['target']}_{row['ID']}"
    dst_path = os.path.join(incorrect_path, dst_filename)
    shutil.copy(src_path, dst_path)

# 클래스별 맞춘 개수와 틀린 개수 집계
results = valid_df.groupby('target')['correct'].value_counts().unstack().fillna(0)
results.columns = ['Incorrect', 'Correct']

# 클래스별 맞춘 개수와 틀린 개수 시각화
plt.figure(figsize=(10, 6))
results.plot(kind='bar', stacked=True)
plt.title('Number of Correct and Incorrect Predictions per Class')
plt.xlabel('Class')
plt.ylabel('Number of Predictions')
plt.legend(title='Prediction')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(weight_path, 'class_prediction_results.png'))
plt.close()

<Figure size 1000x600 with 0 Axes>

In [24]:
# 클래스별 상세 통계
results_detailed = valid_df.groupby('target').agg(
    correct_count=('correct', 'sum'),
    incorrect_count=('correct', lambda x: len(x) - x.sum()),
    total_count=('correct', 'count')
)
results_detailed['accuracy'] = results_detailed['correct_count'] / results_detailed['total_count']

# 전체 정확도 및 F1 점수 계산
overall_accuracy = accuracy_score(valid_df['target'], submission_df['target'])
f1_macro = f1_score(valid_df['target'], submission_df['target'], average='macro')

# 결과 출력
print("각 클래스별 맞춘 개수, 틀린 개수 및 정확도:")
print(results_detailed)
print(f"\n전체 정확도: {overall_accuracy:.4f}")
print(f"F1 매크로 점수: {f1_macro:.4f}")

각 클래스별 맞춘 개수, 틀린 개수 및 정확도:
        correct_count  incorrect_count  total_count  accuracy
target                                                       
0                 200                1          201  0.995025
1                  88                1           89  0.988764
2                 200                0          200  1.000000
3                 174               23          197  0.883249
4                 193                7          200  0.965000
5                 200                0          200  1.000000
6                 199                1          200  0.995000
7                 184               15          199  0.924623
8                 200                0          200  1.000000
9                 200                0          200  1.000000
10                200                1          201  0.995025
11                193                5          198  0.974747
12                200                0          200  1.000000
13                151                3     

In [25]:
# 결과를 CSV 파일로 저장
results_detailed.to_csv(os.path.join(weight_path, 'class_prediction_stats.csv'))

In [26]:
# 정답과 예측 비교하여 오답률 계산
valid_df['correct'] = valid_df['target'] == submission_df['target']
class_counts = valid_df['target'].value_counts()
incorrect_counts = valid_df[~valid_df['correct']]['target'].value_counts()

# 오답률 계산
incorrect_rates = incorrect_counts / class_counts
incorrect_rates = incorrect_rates.fillna(0.0)  # 없는 클래스는 0으로 설정

# 가중치 정규화 (오답률 자체를 가중치로 사용)
weights = incorrect_rates / incorrect_rates.sum()

# 없는 클래스의 가중치를 기본값으로 설정
weights = weights.replace(0.0, weights.max())

# 가중치를 tensor로 변환
weights_tensor = torch.tensor(weights.sort_index().values, dtype=torch.float32)

print("클래스별 가중치:")
print(weights_tensor)

클래스별 가중치:
tensor([0.0122, 0.0276, 0.2869, 0.2869, 0.0860, 0.2869, 0.0123, 0.1852, 0.2869,
        0.2869, 0.0122, 0.0621, 0.2869, 0.0479, 0.2676, 0.2869, 0.2869])


In [27]:
# import cv2
# import torch
# import numpy as np
# from torch.nn import functional as F

# import sys
# sys.path.append("../")
# from data.augmentation import batch_transform
# from utils.test_util import load_model

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model = load_model(model_path="/home/pervinco/upstage-cv-classification-cv7/runs/best_9564/weights/best.pth",
#                    model_name="tf_efficientnetv2_xl.in21k",
#                    num_classes=17,
#                    device=device)

# transform = batch_transform(384, 384)
# image = cv2.imread("/home/pervinco/upstage-cv-classification-cv7/notebook/denoised.png")
# x = transform(image=image)['image'].unsqueeze(0).to(device)

# y_pred = model(x)
# prob = F.softmax(y_pred, dim=1)
# prob = prob.detach().cpu().numpy()
# target = np.argmax(prob)

# print(target)